In [2]:
from astropy.coordinates import SkyCoord, Galactocentric, Galactic
from astropy import units as u
stars = SkyCoord(l=[0]*u.deg,
                 b=[0]*u.deg,
                 distance=[0]*u.pc,
                 frame=Galactic)
stars = stars.transform_to(Galactocentric)
stars.x, stars.y, stars.z

(<Quantity [-8121.97336612] pc>, <Quantity [0.] pc>, <Quantity [20.8] pc>)

In [3]:
from astropy.table import Table
from astropy.coordinates import SkyCoord, Galactocentric, Galactic

from astropy import units as u

c = [
    "SOURCE", "CEPHEID_ID", "L", "B", "DIST", "dDIST", "AGE", "PERIOD", "MODE",
    "I1", "dI1", "I2", "dI2", "I3", "dI3", "I4", "dI4", "W1", "dW1", "W2", "dW2",
    "W3", "dW3", "W4", "dW4", "A_Ks", "SOUxRCE"
]

cep = Table.read("Data_Table_1.dat", format='ascii', 
                 names=c,
                )
mask_positivedist = cep['DIST']>0
cep = cep[mask_positivedist]
cep['SkyCoord_galactic'] = SkyCoord(
    l=cep['L']*u.deg,
    b=cep['B']*u.deg,
    distance=cep['DIST']*u.pc,
    frame='galactic'
)
cep['SkyCoord_galactocentric'] = cep['SkyCoord_galactic'].transform_to(Galactocentric)
cepheids = cep['SkyCoord_galactocentric']

In [29]:
cl = Table.read("Clusters_from_dias_and_a99.txt", format='ascii.ecsv')
cl['SkyCoord'] = SkyCoord(
    ra=cl['RA_ICRS'],
    dec=cl['DE_ICRS'],
    frame='icrs'
)

In [6]:
def format_galactic_coords(coord):
    base = coord[0]  # The 'G' prefix
    if '+' in coord:
        first_num, second_num = coord[1:].split('+')
        second_num = second_num.replace("‡", "")
        second_num = second_num.replace("†", "")
        sign = '+'
    elif '−' in coord or '-' in coord:  # Checking for both hyphen and minus sign
        
        first_num, second_num = coord[1:].replace('−', '-').split('-')
        second_num = second_num.replace("‡", "")
        second_num = second_num.replace("†", "")
        sign = '-'
    else:
        raise ValueError("Invalid coordinate format")
    formatted_first = f"{float(first_num):05.1f}"
    formatted_second = f"{float(second_num):04.1f}"
    final = f"{base}{formatted_first}{sign}{formatted_second}"
    return final
from astropy.table import Table, join
nobs = Table.read("nobs.csv", format='ascii.tab')
nobs['SNR'] = [format_galactic_coords(coord) for coord in nobs['_1']]


In [5]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from astropy.table import Table, join, Column


# Query the Vizier catalog
Vizier.ROW_LIMIT = -1
# Assuming you have already retrieved the catalog and stored it in the 'green' variable
green = Vizier.get_catalogs(catalog="VII/284/snrs")[0]
green['RAJ2000'] = Column(green['RAJ2000'].data, name='RAJ2000')
green['DEJ2000'] = Column(green['DEJ2000'].data, name='DEJ2000')
newgreen = join(nobs,green, keys="SNR")

# Now you can access the columns as normal columns
newgreen['SkyCoord'] = SkyCoord(
    ra=newgreen['RAJ2000'],
    dec=newgreen['DEJ2000'],
    distance=newgreen['nobs'],
    frame='icrs',
    unit=(u.hour, u.deg, u.kpc)
)
green['SkyCoord'] = SkyCoord(
    ra=green['RAJ2000'],
    dec=green['DEJ2000'],
    frame='icrs',
    unit=(u.hour, u.deg)
)
green['SkyCoord_galactic'] = green['SkyCoord'].transform_to(Galactic)
newgreen['SkyCoord_galactocentric'] = newgreen['SkyCoord'].transform_to(Galactocentric)
psrs['SkyCoord_galactocentric'] = psrs['SkyCoord'].transform_to(Galactocentric)

NameError: name 'psrs' is not defined

In [63]:
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib qt
image = Image.open('mw.jpg')
radius = 21.200
x_min, x_max = -radius, radius  # Example: -10 kpc to 10 kpc
y_min, y_max = -radius, radius  # Example: -10 kpc to 10 kpc

star_x = [0,]  # Replace with actual x coordinates
star_y = [8.122,]  # Replace with actual y coordinates

plt.rcParams["font.family"] = "Palatino"
plt.rcParams["font.size"] = 44



fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(image, extent=[x_min, x_max, y_min, y_max], alpha=0.7)
ax.scatter(star_x, star_y, color='yellow', s=100, edgecolor='red', zorder=6)

dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
psrs = dias2021[maskage & maskplx]
psrs['SkyCoord'] = SkyCoord(
    ra=psrs['RA_ICRS'],
    dec=psrs['DE_ICRS'],
    distance=psrs['Dist']
)
psrs['SkyCoord_galactocentric'] = psrs['SkyCoord'].transform_to(Galactocentric)
snrs = psrs['SkyCoord_galactocentric']

ax.scatter(snrs.y/1000,-snrs.x/1000, alpha=0.8, s=20,color='blue', edgecolor='cyan')
        #    ,edgecolor='k')
# Set labels and title
ax.set_xlabel('Galactic X (kpc)')
ax.set_ylabel('Galactic Y (kpc)')
# ax.set_title('Star Positions on the Milky Way')
ax.grid(alpha=0.5)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.invert_yaxis()
ax.invert_xaxis()
plt.subplots_adjust(
    top=1.0,      # Border from the top of the figure
    bottom=0.129, # Border from the bottom of the figure
    left=0.124,   # Border from the left of the figure
    right=0.991,  # Border from the right of the figure
    hspace=0.0,   # Horizontal space between subplots
    wspace=0.0    # Vertical space between subplots
)
# plt.tight_layout()
plt.show()


In [32]:
snrs.x

<Quantity [ -7854.26107905,  -8412.14548109,  -8438.10258824,
            -9320.41809508,  -8608.26664003,  -8604.89879851,
            -8609.13749938,  -8713.97557302,  -7502.91995576,
            -7664.75326253, -10020.61095334,  -8495.69349911,
            -8420.99776289,  -8517.55120291,  -7224.89334886,
            -6933.54367151,  -8442.13885008,  -8992.66001224,
            -9071.87291251,  -9109.1594839 ,  -7353.79280286,
            -8011.23636885,  -6410.06337805,  -6746.06907952,
            -6224.36590971,  -7256.99670036,  -8420.70861834,
            -5663.28809488,  -8504.76133884,  -8383.40583264,
            -7946.54915059,  -7793.29407699,  -8177.54789072,
            -7214.52737006,  -9631.90244487, -10738.92278302,
           -10300.08540843,  -7099.57592161,  -8831.20113359,
            -9695.6469896 ,  -9883.98392136,  -9841.79178465,
            -6426.23773448,  -7903.70484145,  -7879.81213654,
            -9011.43262742,  -8036.60178904,  -7814.61550863,
        

In [9]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from astropy.table import Table, join, Column
from astropy.coordinates import SkyCoord, Galactocentric, Galactic
%matplotlib qt
from astropy import units as u
plt.rcParams["font.size"] = 20

# Query the Vizier catalog
Vizier.ROW_LIMIT = -1
# Assuming you have already retrieved the catalog and stored it in the 'green' variable
green = Vizier.get_catalogs(catalog="VII/284/snrs")[0]
green['RAJ2000'] = Column(green['RAJ2000'].data, name='RAJ2000')
green['DEJ2000'] = Column(green['DEJ2000'].data, name='DEJ2000')
green['SkyCoord'] = SkyCoord(
    ra=green['RAJ2000'],
    dec=green['DEJ2000'],
    frame='icrs',
    unit=(u.hour, u.deg)
)
green['SkyCoord_galactic'] = green['SkyCoord'].transform_to(Galactic)
# Load the image
image = Image.open('galactic.png')

# Create the figure
fig = plt.figure(figsize=(20, 10))

# Create the background image axis
ax_img = fig.add_axes([0, 0, 1, 1], label='background')
ax_img.imshow(image, alpha=0.8)
ax_img.axis('off')  # Hide the axes for the image

# Create the Aitoff projection on top of the image
ax_plot = fig.add_subplot(111, projection="aitoff", frame_on=False)

# Set the background color of the projection to transparent
# ax_plot.patch.set_alpha(0.4)


# Scatter plot overlay
ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
                green['SkyCoord_galactic'].b.radian, c="red", s=green['MajDiam'], zorder=4, alpha=0.2)
# diameters
ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
                green['SkyCoord_galactic'].b.radian, c="red", s=7, zorder=5)

# Add grid
ax_plot.grid(True)
# Adjust the subplot to fill more of the figure
plt.subplots_adjust(top=0.88,
                    bottom=0.12,
                    left=0.166,
                    right=0.833,
                    hspace=0.2,
                    wspace=0.2
                    )

plt.show()

In [34]:
from astroquery.hips2fits import hips2fits
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
from astropy import wcs as astropy_wcs
# Create a new WCS astropy object
w = astropy_wcs.WCS(header={
    'NAXIS1': 2000,         # Width of the output fits/image
    'NAXIS2': 1000,         # Height of the output fits/image
    'WCSAXES': 2,           # Number of coordinate axes
    'CRPIX1': 1000.0,       # Pixel coordinate of reference point
    'CRPIX2': 500.0,        # Pixel coordinate of reference point
    'CDELT1': -0.18,        # [deg] Coordinate increment at reference point
    'CDELT2': 0.18,         # [deg] Coordinate increment at reference point
    'CUNIT1': 'deg',        # Units of coordinate increment and value
    'CUNIT2': 'deg',        # Units of coordinate increment and value
    'CTYPE1': 'GLON-MOL',   # galactic longitude, Mollweide's projection
    'CTYPE2': 'GLAT-MOL',   # galactic latitude, Mollweide's projection
    'CRVAL1': 0.0,          # [deg] Coordinate value at reference point
    'CRVAL2': 0.0,          # [deg] Coordinate value at reference point
})
hips = 'CDS/P/Mellinger/color'

result = hips2fits.query_with_wcs(
   hips=hips,
   wcs=w,
   get_query_payload=False,
   format='jpg',
   min_cut=0.5,
   max_cut=99.5,
   cmap=Colormap('viridis'),
)
im = plt.imshow(result)
plt.show() 

In [11]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from astropy.table import Table, join, Column
from astropy.coordinates import SkyCoord, Galactocentric, Galactic
%matplotlib qt
from astropy import units as u
plt.rcParams["font.size"] = 20

# Query the Vizier catalog
Vizier.ROW_LIMIT = -1
# SNRS
green = Vizier.get_catalogs(catalog="VII/284/snrs")[0]
green['RAJ2000'] = Column(green['RAJ2000'].data, name='RAJ2000')
green['DEJ2000'] = Column(green['DEJ2000'].data, name='DEJ2000')
green['SkyCoord'] = SkyCoord(
    ra=green['RAJ2000'],
    dec=green['DEJ2000'],
    frame='icrs',
    unit=(u.hour, u.deg)
)
green['SkyCoord_galactic'] = green['SkyCoord'].transform_to(Galactic)



#Pulsars
from psrqpy import QueryATNF
query = QueryATNF(version='2.3.0')
psrs = query.table
mask_dist = ~psrs['DIST'].mask
psrs = psrs[mask_dist]
psrs['SkyCoord'] = SkyCoord(
    ra=psrs['RAJD'],
    dec=psrs['DECJD'],
    distance=psrs['DIST']
)
psrs['SkyCoord_galactic'] = psrs['SkyCoord'].transform_to(Galactic)

# Load the image
image = Image.open('galactic.png')

# Create the figure
fig = plt.figure(figsize=(20, 10))

# Create the background image axis
ax_img = fig.add_axes([0, 0, 1, 1], label='background')
ax_img.imshow(image, alpha=0.8)
ax_img.axis('off')  # Hide the axes for the image

# Create the Aitoff projection on top of the image
ax_plot = fig.add_subplot(111, projection="aitoff", frame_on=False)
# Scatter PSRs
ax_plot.scatter(-psrs['SkyCoord_galactic'].l.wrap_at('180d').radian, 
                psrs['SkyCoord_galactic'].b.radian, c="cyan", s=5,)
# Set the background color of the projection to transparent
# ax_plot.patch.set_alpha(0.4)


# # Scatter SNR diameters
# ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
#                 green['SkyCoord_galactic'].b.radian, c="red", s=green['MajDiam'], zorder=4, alpha=0.2)
# # Scatter SNRs
# ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
#                 green['SkyCoord_galactic'].b.radian, c="red", s=7, zorder=5)


# Add grid
ax_plot.grid(True)
# Adjust the subplot to fill more of the figure
plt.subplots_adjust(top=0.88,
                    bottom=0.12,
                    left=0.166,
                    right=0.833,
                    hspace=0.2,
                    wspace=0.2
                    )

plt.show()

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from astropy.table import Table, join, Column
from astropy.coordinates import SkyCoord, Galactocentric, Galactic
%matplotlib qt
from astropy import units as u
plt.rcParams["font.size"] = 20

# Query the Vizier catalog
Vizier.ROW_LIMIT = -1
# SNRS
green = Vizier.get_catalogs(catalog="VII/284/snrs")[0]
green['RAJ2000'] = Column(green['RAJ2000'].data, name='RAJ2000')
green['DEJ2000'] = Column(green['DEJ2000'].data, name='DEJ2000')
green['SkyCoord'] = SkyCoord(
    ra=green['RAJ2000'],
    dec=green['DEJ2000'],
    frame='icrs',
    unit=(u.hour, u.deg)
)
green['SkyCoord_galactic'] = green['SkyCoord'].transform_to(Galactic)



dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
psrs = dias2021[maskage & maskplx]
psrs['SkyCoord'] = SkyCoord(
    ra=psrs['RA_ICRS'],
    dec=psrs['DE_ICRS'],
    distance=psrs['Dist']
)

psrs['SkyCoord_galactic'] = psrs['SkyCoord'].transform_to(Galactic)

# Load the image
image = Image.open('galactic.png')

# Create the figure
fig = plt.figure(figsize=(20, 10))

# Create the background image axis
ax_img = fig.add_axes([0, 0, 1, 1], label='background')
ax_img.imshow(image, alpha=0.8)
ax_img.axis('off')  # Hide the axes for the image

# Create the Aitoff projection on top of the image
ax_plot = fig.add_subplot(111, projection="aitoff", frame_on=False)
# Scatter PSRs
ax_plot.scatter(-psrs['SkyCoord_galactic'].l.wrap_at('180d').radian, 
                psrs['SkyCoord_galactic'].b.radian, c="deepskyblue", s=20,)
ax_plot.scatter(-psrs['SkyCoord_galactic'].l.wrap_at('180d').radian, 
                psrs['SkyCoord_galactic'].b.radian, c="deepskyblue", s=psrs['r50'].value, alpha=0.2
                )
# ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
#                 green['SkyCoord_galactic'].b.radian, c="red", s=7, zorder=5)
# Set the background color of the projection to transparent
# ax_plot.patch.set_alpha(0.4)


# # Scatter SNR diameters
# ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
#                 green['SkyCoord_galactic'].b.radian, c="red", s=green['MajDiam'], zorder=4, alpha=0.2)
# # Scatter SNRs
# ax_plot.scatter(-green['SkyCoord_galactic'].l.wrap_at('180d').radian, 
#                 green['SkyCoord_galactic'].b.radian, c="red", s=7, zorder=5)


# Add grid
ax_plot.grid(True)
# Adjust the subplot to fill more of the figure
plt.subplots_adjust(top=0.88,
                    bottom=0.12,
                    left=0.166,
                    right=0.833,
                    hspace=0.2,
                    wspace=0.2
                    )

plt.show()

In [4]:
2*np.arctan(1/2)*180/np.pi

53.13010235415598